# 🔍 Day 2 실습 2: Naive RAG 실패 케이스 상세 분석

## 🎯 학습 목표
- **5가지 실패 케이스**로 Naive RAG의 다양한 한계 체험
- **검색 과정 시각화**로 왜 실패하는지 직관적으로 이해
- **해결 힌트 3가지**로 Advanced RAG 기법 예습

## 💡 01번 복습
- ✅ 단순 질문: 성공
- ❌ 복합 질문: 실패
- 🤔 왜?: 동음이의어, 멀티홉, 노이즈

## 🚀 02번에서 할 것
1. 5가지 다양한 실패 케이스 테스트
2. 검색 과정 상세 분석 (유사도 점수, Top-K)
3. 해결 방법 힌트 제시 (03-05번 예고)

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers pandas matplotlib seaborn

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
import platform
if platform.system() == 'Darwin':
    plt.rcParams['font.family'] = ['AppleGothic']
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = ['Malgun Gothic']
else:
    plt.rcParams['font.family'] = ['NanumGothic']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 준비 완료!")

## 📚 01번과 동일한 문서 & 벡터스토어

일관성을 위해 01번과 똑같은 환경을 사용합니다:

In [ ]:
# 01번과 동일한 문서 (8개)
pastel_docs = [
    {
        "content": """파스텔 호텔 예약 안내
서울 강남구에 위치한 파스텔 호텔입니다.
- 예약 변경: 투숙 3일 전까지 가능 (위약금 없음)
- 주말 요금: 평일 대비 30% 할인""",
        "type": "hotel"
    },
    {
        "content": """파스텔 카페 메뉴
홍대 파스텔 카페는 디저트로 유명합니다.
- 예약: 단체 예약만 가능 (10명 이상)
- 주말: 마카롱 만들기 체험 (14:00-16:00)""",
        "type": "cafe"
    },
    {
        "content": """파스텔 소극장 공연 안내
대학로 파스텔 소극장에서 뮤지컬을 상영합니다.
- 티켓 예약: 인터파크, 현장 구매 가능
- 주말: 토요일 15:00 추가 공연""",
        "type": "theater"
    },
    {
        "content": """국립현대미술관 전시 정보
서울 국립현대미술관 덕수궁관 전시 안내입니다.
- 주말 관람 팁: 토요일 14:00-16:00 혼잡, 일요일 오전 10:00-12:00 한적함""",
        "type": "museum"
    },
    {
        "content": """서울 여행 블로그
미술관은 주말 오전이 베스트 타이밍.
예약은 미리미리! 특히 숙소는 필수.""",
        "type": "blog"
    },
    {
        "content": """서울역 교통 안내
KTX, 지하철 1/4호선 환승 가능.
주말 막차 시간: 자정까지 운행""",
        "type": "transport"
    },
    {
        "content": """주말 날씨 예보
이번 주말 서울 날씨는 맑음.
외출하기 좋은 날씨입니다.""",
        "type": "weather"
    },
    {
        "content": """예약 플랫폼 가이드
네이버, 카카오 예약 시스템 비교.
주말 예약은 평일보다 2배 빠르게 마감됩니다.""",
        "type": "guide"
    }
]

# Document 객체로 변환
documents = [Document(page_content=doc["content"], metadata={"type": doc["type"]}) 
             for doc in pastel_docs]

# 벡터스토어 구축
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents, embeddings)

print("✅ 벡터스토어 준비 완료!")
print(f"   문서 수: {len(documents)}개")

## 🧪 Part 1: 5가지 실패 케이스 테스트

다양한 질문으로 Naive RAG의 한계를 체험해봅시다:

In [ ]:
# 5가지 실패 시나리오
test_cases = [
    {
        "query": "파스텔 예약하고 싶어요",
        "challenge": "🤷 동음이의어",
        "expected": "hotel? cafe? theater?"
    },
    {
        "query": "파스텔 주말 이벤트 있나요?",
        "challenge": "🎭 cafe vs theater",
        "expected": "cafe(체험) or theater(공연)?"
    },
    {
        "query": "강남에 있는 파스텔 찾아줘",
        "challenge": "📍 지역 정보",
        "expected": "hotel만 강남"
    },
    {
        "query": "파스텔에서 체험할 수 있는 거 있어?",
        "challenge": "🎨 cafe 특화",
        "expected": "cafe(마카롱 만들기)"
    },
    {
        "query": "내일 저녁에 파스텔 공연 보고, 주말엔 호텔 체크인 날짜 바꾸고, 카페에서 체험 수업도 받고 싶어",
        "challenge": "🔗 멀티홉 (01번)",
        "expected": "theater + hotel + cafe"
    }
]

print("🧪 5가지 실패 케이스 테스트")
print("=" * 80)
print()

# 각 케이스 실행
for i, case in enumerate(test_cases, 1):
    print(f"### 케이스 {i}: {case['challenge']}")
    print(f"❓ 질문: {case['query']}")
    print(f"💡 기대: {case['expected']}")
    print()
    
    # Top-3 검색
    results = vectorstore.similarity_search_with_score(case["query"], k=3)
    
    print("🔍 검색 결과 (Top-3):")
    for j, (doc, score) in enumerate(results, 1):
        doc_type = doc.metadata.get("type", "unknown")
        print(f"  {j}. [{doc_type.upper()}] 유사도: {1-score:.3f}")
    
    print()
    print("-" * 80)
    print()

## 📊 실패 패턴 요약

위 결과에서 발견한 공통 문제점:

In [ ]:
print("📊 실패 패턴 분석")
print()
print("1️⃣ 동음이의어 문제")
print("   - '파스텔' → hotel, cafe, theater 모두 유사도 높음")
print("   - 맥락 없이는 구분 불가")
print()
print("2️⃣ 노이즈 문서 간섭")
print("   - '주말', '예약' 같은 일반 키워드")
print("   - weather, guide, blog 같은 관련 없는 문서도 상위권")
print()
print("3️⃣ 멀티홉 추론 실패")
print("   - 3개 정보 필요 → Top-3로 부족")
print("   - 정보 연결 능력 없음")
print()
print("4️⃣ 메타데이터 미활용")
print("   - '강남' → type, location 필터 필요")
print("   - 벡터 검색만으로는 한계")

## 📈 Part 2: 검색 과정 상세 분석

### 벡터 유사도 시각화

"파스텔 예약하고 싶어요" 질문에 대한 모든 문서의 유사도 점수:

In [ ]:
# 모든 문서 검색 (Search all documents)
query = "파스텔 예약하고 싶어요"
all_results = vectorstore.similarity_search_with_score(query, k=8)

# 문서 타입과 점수 추출 (Extract document types and scores)
doc_names = [doc.metadata['type'] for doc, _ in all_results]
scores = [1-score for _, score in all_results]  # 1-distance = similarity

# 막대그래프 생성 (Create bar chart)
plt.figure(figsize=(10, 6))
colors = ['red' if i < 3 else 'gray' for i in range(len(scores))]
plt.barh(doc_names, scores, color=colors)
plt.xlabel("Similarity Score (higher is more similar)")  # 유사도 점수 (높을수록 유사)
plt.title(f"Search Results")  # 검색 결과
plt.axvline(x=scores[2], color='red', linestyle='--', linewidth=2, label='Top-3 Cutoff')  # Top-3 기준선
plt.legend()
plt.tight_layout()
plt.show()

print("📊 상세 분석:")
print()
print("🔴 Top-3 검색 결과 (빨간색):")
for i in range(3):
  print(f"   {i+1}. {doc_names[i]}: {scores[i]:.3f}")
print()

print("⚠️ 발견된 문제점:")
print()
print("1️⃣ 동음이의어 구분 실패:")
print(f"   - hotel ({scores[0]:.3f}), cafe ({scores[1]:.3f}), theater ({scores[2]:.3f})")
print(f"   - 점수 차이: {abs(scores[0] - scores[2]):.3f} (거의 동일!)")
print("   → '파스텔'이 호텔/카페/소극장 중 무엇인지 구분 불가")
print()

print("2️⃣ 노이즈 문서 간섭:")
print("   - weather, transport, museum도 높은 유사도")
print(f"   - museum 점수: {scores[doc_names.index('museum')]:.3f}")
print(f"   - weather 점수: {scores[doc_names.index('weather')]:.3f}")
print("   → '예약', '주말' 같은 일반 키워드 때문")
print()

print("3️⃣ Top-3 제한의 문제:")
print(f"   - Top-3 기준선: {scores[2]:.3f}")
print(f"   - 4위 문서 (blog): {scores[3]:.3f}")
print(f"   - 차이: {abs(scores[2] - scores[3]):.3f}")
print("   → Top-3를 조금만 벗어나도 필요한 정보 누락 가능")
print()

print("💡 핵심 인사이트:")
print("   ✗ 벡터 검색만으로는 동음이의어 구분 불가")
print("   ✗ 일반 키워드가 많은 문서가 노이즈로 작용")
print("   ✗ 고정된 Top-K는 질문 복잡도를 고려하지 못함")
print()
print("   ✓ 맥락 정보 필요 (메타데이터 필터링)")
print("   ✓ 질문 분해 필요 (Multi-Query)")
print("   ✓ 키워드 매칭 추가 필요 (Hybrid Search)")

## 🔢 Top-K 변화 실험

k값을 바꾸면 어떻게 될까요?

In [ ]:
# 멀티홉 질문으로 테스트
query = "내일 저녁에 파스텔 공연 보고, 주말엔 호텔 체크인 날짜 바꾸고, 카페에서 체험 수업도 받고 싶어"
needed = ["theater", "hotel", "cafe"]

results_by_k = []
for k in [1, 2, 3, 5, 8]:
    results = vectorstore.similarity_search(query, k=k)
    found = [doc.metadata['type'] for doc in results]
    has_all = all(need in found for need in needed)
    
    results_by_k.append({
        "k": k,
        "검색됨": ", ".join(found[:5]),  # 최대 5개만 표시
        "필요한 문서 포함": f"{sum(need in found for need in needed)}/3",
        "완성도": "✅" if has_all else "❌"
    })

df = pd.DataFrame(results_by_k)
print("📊 Top-K 변화에 따른 결과:\n")
print(df.to_string(index=False))
print()
print("💡 발견:")
print("   - k=1,2,3: 필요한 문서 모두 못 찾음")
print("   - k=5 이상: 성공 가능 (하지만 노이즈 증가)")
print("   - 단순히 k 늘린다고 해결 안됨!")

## 👤 Part 3: 사람 vs Naive RAG 비교

같은 질문을 사람과 RAG는 어떻게 다르게 처리할까요?

In [ ]:
query = "내일 저녁에 파스텔 공연 보고, 주말엔 호텔 체크인 날짜 바꾸고, 카페에서 체험 수업도 받고 싶어"

print("🎯 질문:")
print(f"   {query}")
print()
print("=" * 80)
print()

print("👤 사람의 추론 과정:")
print()
print("  1️⃣ 질문 이해")
print("     - '공연' → 소극장 정보 필요")
print("     - '호텔 체크인' → 호텔 정보 필요")
print("     - '카페 체험' → 카페 정보 필요")
print()
print("  2️⃣ 엔티티 해결")
print("     - 파스텔 공연 = theater")
print("     - 파스텔 호텔 = hotel")
print("     - 파스텔 카페 = cafe")
print()
print("  3️⃣ 정보 수집")
print("     - 3개 문서 모두 검색 필요")
print("     - 각각의 정보 확인")
print()
print("  4️⃣ 답변 생성")
print("     - 3개 정보를 통합")
print("     - 완전한 답변 제공")
print()
print("=" * 80)
print()

print("🤖 Naive RAG의 처리:")
print()
print("  1️⃣ 질문 이해")
print("     - 전체를 하나의 쿼리로 처리")
print("     - 키워드: '파스텔', '공연', '호텔', '카페', '주말'")
print()
print("  2️⃣ 엔티티 해결")
print("     - 파스텔 = 모호함 (동음이의어 미해결)")
print("     - 맥락 이해 부족")
print()
print("  3️⃣ 정보 수집")
print("     - 벡터 유사도로 Top-3만 검색")
results = vectorstore.similarity_search(query, k=3)
for i, doc in enumerate(results, 1):
    print(f"     - {i}. {doc.metadata['type']}")
print()
print("  4️⃣ 답변 생성")
print("     - 검색된 정보만으로 답변")
print("     - 불완전한 정보 → 실패")
print()
print("=" * 80)

## 💡 Part 4: 해결 힌트 3가지

Advanced RAG 기법으로 어떻게 해결할 수 있을까요?

### 힌트 1: Multi-Query (03번에서 배울 것)

In [ ]:
print("💡 해결 아이디어 1: 질문 분해 (Multi-Query)")
print()
print("원래 질문:")
print("  '파스텔 공연 + 호텔 체크인 + 카페 체험'")
print()
print("분해:")
sub_queries = [
    "파스텔 소극장 공연 정보",
    "파스텔 호텔 체크인 변경 규정",
    "파스텔 카페 체험 수업"
]
for i, sq in enumerate(sub_queries, 1):
    print(f"  {i}. '{sq}'")
print()
print("각각 검색:")
for i, sq in enumerate(sub_queries, 1):
    result = vectorstore.similarity_search(sq, k=1)
    print(f"  Query {i} → {result[0].metadata['type']}")
print()
print("결과: ✅ 3개 정보 모두 확보!")
print()
print("👉 03번 노트북에서 Multi-Query Retrieval 실습")

### 힌트 2: Metadata Filtering (04번에서 배울 것)

In [ ]:
print("💡 해결 아이디어 2: 메타데이터 필터링")
print()
print("문제 상황:")
query = "강남에 있는 파스텔 찾아줘"
print(f"  질문: '{query}'")
results = vectorstore.similarity_search(query, k=3)
print("  검색 결과:", [doc.metadata['type'] for doc in results])
print("  문제: hotel, cafe, theater 모두 검색됨")
print()
print("해결 방법:")
print("  만약 메타데이터에 location='강남' 정보가 있다면?")
print("  검색 조건: 'type=hotel AND location=강남'")
print("  결과: hotel만 → 동음이의어 해결!")
print()
print("다른 예시:")
print("  '파스텔 호텔 예약' → filter: type='hotel'")
print("  '파스텔 카페 메뉴' → filter: type='cafe'")
print()
print("👉 04번 노트북에서 Metadata Filtering 실습")

### 힌트 3: Hybrid Search (05번에서 배울 것)

In [ ]:
print("💡 해결 아이디어 3: Hybrid Search (벡터 + 키워드)")
print()
print("현재 (벡터 검색만):")
print("  장점: 의미 기반 검색 가능")
print("  단점: 일반 키워드('주말', '예약')에 약함")
print("       노이즈 문서(weather, guide)도 상위권")
print()
print("키워드 검색 추가:")
print("  - '예약 변경' 정확히 매칭")
print("  - '체험 수업' 정확히 매칭")
print("  - BM25 알고리즘 사용")
print()
print("벡터 + 키워드 결합 (Hybrid):")
print("  - 의미 유사도: 0.7")
print("  - 키워드 매칭: 0.8")
print("  - 최종 점수: (0.7 + 0.8) / 2 = 0.75")
print()
print("결과:")
print("  - 의미도 잡고 정확도도 잡음")
print("  - Recall(재현율) 향상")
print("  - 노이즈 문서 필터링")
print()
print("👉 05번 노트북에서 Hybrid Search + Re-ranking 실습")

## 🎓 핵심 교훈 정리

### 01번에서 본 것
- ✅ 단순 질문: 성공
- ❌ 복합 질문: 실패
- 🤔 왜?: 3가지 이유 (동음이의어, 멀티홉, 노이즈)

### 02번에서 본 것
- 🧪 **5가지 실패 케이스**: 동음이의어, 지역 정보, 체험 정보, 주말 이벤트, 멀티홉
- 📊 **검색 과정 분석**: 유사도 시각화, Top-K 실험
- 👤 **사람 vs RAG**: 추론 과정 비교
- 💡 **해결 힌트 3가지**: Multi-Query, Metadata Filtering, Hybrid Search

### Naive RAG의 근본 문제
```
💭 사용자 의도: "여러 정보를 종합해서 답변"
🤖 Naive RAG: "키워드로 유사한 문서 3개만 검색"
```

### 03-06번에서 할 것
- **03번**: Multi-Query로 질문 분해 → 멀티홉 해결
- **04번**: Metadata Filtering → 동음이의어 해결
- **05번**: Hybrid Search + Re-ranking → 정확도 향상
- **06번**: Modular RAG → 통합 시스템

---

**🚀 다음 실습 예고**

03번 노트북에서 Multi-Query Retrieval로 복합 질문을 분해하고, 각각 검색하여 통합하는 방법을 배워봅시다!

In [ ]:
# 🎉 실습 완료!
print("🎉 02번 Naive RAG 실패 분석 완료!")
print()
print("💡 오늘 배운 것:")
print("  ✅ 5가지 다양한 실패 케이스")
print("  ✅ 검색 과정 상세 분석 (시각화)")
print("  ✅ 사람 vs RAG 추론 과정 비교")
print("  ✅ 해결 힌트 3가지 (Multi-Query, Metadata, Hybrid)")
print()
print("🚀 다음 단계:")
print("  → 03번: Multi-Query Retrieval")
print("  → 04번: Metadata Filtering")
print("  → 05번: Hybrid Search + Re-ranking")